**Install Dependencies**

In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,983 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
!pip install pytesseract

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


**Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import cv2
import pytesseract

from glob import glob
from tqdm import tqdm
import os

**Import Data**

In [ ]:
img = cv2.imread("/content/drive/MyDrive/CardData/052.jpeg")

In [ ]:
# Save path to all business card to imgPaths
imgPaths = glob("/content/drive/MyDrive/CardData/*.jpeg")
imgPaths

In [ ]:
imgPath = imgPaths[0]
imgPath

'/content/drive/MyDrive/CardData/011.jpeg'

In [ ]:
complete_data = pd.DataFrame(columns = ["id", "text"])

for imgPath in tqdm(imgPaths, desc = " BusinessCard"):

  # Extract file name from business card eg. jpeg
  _, filname = os.path.split(imgPath)

  # Extract data and text
  image = cv2.imread(imgPath)
  data = pytesseract.image_to_data(image)

  # Convert extracted data to dataframe
  dataList = list(map(lambda x: x.split("\t"), data.split("\n")))
  df = pd.DataFrame(dataList[1:], columns = dataList[0])

  # Drop missing values and converting obj to int
  df.dropna(inplace = True)
  df["conf"] = df["conf"].astype(int)

  # Extract only level 5 which has a confidence score >30
  confidence = df.query("conf >= 30")

  # Create a dataframe that has filename and text
  dataset = pd.DataFrame()
  dataset["text"] = confidence["text"]
  dataset["id"] = filname

  # Concatenate new dataframe with existing one
  complete_data = pd.concat((complete_data, dataset))

 BusinessCard: 100%|██████████| 293/293 [09:51<00:00,  2.02s/it]


In [ ]:
complete_data

,id,text
4,011.jpeg,a
5,011.jpeg,:
6,011.jpeg,Cell
7,011.jpeg,:
8,011.jpeg,9885050228
...,...,...
52,276.jpeg,(cell)
55,276.jpeg,skulumani@gwu.edu
56,276.jpeg,(email)
58,276.jpeg,shankarkulumani.com


In [ ]:
complete_data.to_csv("MyDataset.csv", index=False)

**Image to Text**

**Text to Data**